Value would be the class whose objects would be the nodes of our imaginary DAG for forward pass and back prop.

In [4]:
import numpy as np
import matplotlib.pyplot as plt
import math
%matplotlib inline

In [7]:
class Value:
    def __init__(self,data,_children=(),_op='',label=''):#using tuple because tuple is immutable.
        self.data=data
        self.grad=0 #derivative of final variable wrt this variable
        #ofc not applicable for leaf nodes which correspond to input.
        #just find prod of the adjacent node if its a * label, else it will be 1.
        self._prev=_children
        self._op=_op
        self.label=label
        self._backward=lambda:None # this is the backward function which will be determined at runtime for each Value object. 
        #by the way we implement it, it would be only for derived nodes and not leaf nodes.
    def __repr__(self):
        return f"Value(data:{self.data})"
    def __add__(self,other):
        out=Value(self.data+other.data,(self,other),'+')
        def _backward(): # this fills in the derivatives of children
            self.grad+=out.grad #using += in case same node is used more than once, eg: b=a+a or (d=a+b and c=a*b) and we don't override the definition and instead
            #sum up the derivatives of all the instances of the node
            other.grad+=out.grad #adding in
        out._backward=_backward
        return out
    def __mul__(self,other):
        out=Value(self.data*other.data,(self,other),'*')
        def _backward():
            self.grad+=other.data*out.grad
            other.grad+=self.data*out.grad
        out._backward=_backward
        return out
    def tanh(self):
        x=self.data
        res = (math.exp(2*x)-1)/(math.exp(2*x)+1)
        out =Value(res,(self,),_op='tanh')
        def _backward():
            self.grad+=1-(out.data**2)
        out._backward=_backward
        return out
    def backward(self):
        self.grad=1
        topo=[]
        def build_topo(node):
            visited=set()
            for child in node._prev:
                if child not in visited:
                    build_topo(child)
            topo.append(node)
        build_topo(self)
        for node in reversed(topo):
            node._backward()

a=Value(5,label='a')
b=Value(3,label='b')
c=a+b
c.label='c'
d=Value(10,label='d')
e=c*d
e.label='e'
e.backward()
print(a.grad)
print(b.grad)
print(c.grad)
print(d.grad)
print(e.grad)

10
10
10
8
1


In [6]:
# inputs x1,x2
x1 = Value(2.0, label='x1')
x2 = Value(0.0, label='x2')
# weights w1,w2
w1 = Value(-3.0, label='w1')
w2 = Value(1.0, label='w2')
# bias of the neuron
b = Value(6.8813735870195432, label='b')
# x1*w1 + x2*w2 + b
x1w1 = x1*w1; x1w1.label = 'x1*w1'
x2w2 = x2*w2; x2w2.label = 'x2*w2'
x1w1x2w2 = x1w1 + x2w2; x1w1x2w2.label = 'x1*w1 + x2*w2'
n = x1w1x2w2 + b; n.label = 'n'
o = n.tanh(); o.label = 'o'
o.grad=1
o._backward()
n.grad

0.4999999999999999

In [5]:
from graphviz import Digraph

def trace(root):
  # builds a set of all nodes and edges in a graph
  nodes, edges = set(), set()
  def build(v):
    if v not in nodes:
      nodes.add(v)
      for child in v._prev:
        edges.add((child, v))
        build(child)
  build(root)
  return nodes, edges

def draw_dot(root):
  dot = Digraph(format='svg', graph_attr={'rankdir': 'LR'}) # LR = left to right
  
  nodes, edges = trace(root)
  for n in nodes:
    uid = str(id(n))
    # for any value in the graph, create a rectangular ('record') node for it
    dot.node(name = uid, label = "{ %s | data %.4f | grad %.4f }" % (n.label, n.data, n.grad), shape='record')
    if n._op:
      # if this value is a result of some operation, create an op node for it
      dot.node(name = uid + n._op, label = n._op)
      # and connect this node to it
      dot.edge(uid + n._op, uid)

  for n1, n2 in edges:
    # connect n1 to the op node of n2
    dot.edge(str(id(n1)), str(id(n2)) + n2._op)

  return dot
draw_dot(c)

ExecutableNotFound: failed to execute WindowsPath('dot'), make sure the Graphviz executables are on your systems' PATH